# 下载嫦娥公开科学数据

In [1]:
# 依赖包
!pip install requests -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 61 kB 2.5 MB/s eta 0:00:011
     |████████████████████████████████| 178 kB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 1.7 MB/s eta 0:00:011
     |████████████████████████████████| 153 kB 1.2 MB/s eta 0:00:01


## 1. 测试网络爬虫

进入[探月工程数据发布与信息服务系统](http://moon.bao.ac.cn/)进行查询，F12找到`proSearch!proSearchList.search`的`GET`请求，在`headers`页面将`GET`部分`copy All`。

以下代码中`GET_copyall_from_firefox`就是复制过来的内容。

In [1]:
import atool

GET_copyall_from_firefox = {"GET":{"scheme":"https","host":"moon.bao.ac.cn","filename":"/proSearch!proSearchList.search","query":{"_dc":"1615632080475","pageNo":"2","pageSize":"10","inputJsonString":"{\"globalLang\":null,\"DM_DTYPE\":\"SCI\",\"DM_TASKID\":\"CE4\",\"DM_ZHNAME\":\"LCAM\",\"DM_GRADE\":\"2A\",\"DM_CBTIMEStart\":\"2019-01-03 00:00:00\",\"DM_CBTIMEend\":\"2019-01-03 23:59:59\",\"order_by\":\"desc\",\"qsList\":\"\",\"qsStart\":\"\",\"qsEnd\":\"\",\"bianhao\":\"\"}","page":"2","start":"10","limit":"10"},"remote":{"Address":"159.226.88.59:443"}}}

print(atool.pretty(GET_copyall_from_firefox))

{
    GET={
        scheme=https
        host=moon.bao.ac.cn
        filename=/proSearch!proSearchList.search
        query={
            _dc=1615632080475
            pageNo=2
            pageSize=10
            inputJsonString={"globalLang":null,"DM_DTYPE":"SCI","DM_TASKID":"CE4","DM_ZHNAME":"LCAM","DM_GRADE":"2A","DM_CBTIMEStart":"2019-01-03 00:00:00","DM_CBTIMEend":"2019-01-03 23:59:59","order_by":"desc","qsList":"","qsStart":"","qsEnd":"","bianhao":""}
            page=2
            start=10
            limit=10
        }
        remote={
            Address=159.226.88.59:443
        }
    }
}



将`GET_copyall_from_firefox`中的`scheme`，`host`，`filename`三个字段合成`url`传入`requests.get()`；将`query`作为`param`传入`requests.get()`。

特别需要注意的是，`query`中的`inputJsonString`是查询参数，该更改的参数包括任务、载荷、数据类型、数据级别、起止时间等。

In [2]:
import requests

url = GET_copyall_from_firefox['GET']['scheme']+'://'+GET_copyall_from_firefox['GET']['host']+GET_copyall_from_firefox['GET']['filename']
param = GET_copyall_from_firefox['GET']['query']

print(url)
print(atool.pretty(param))

r = requests.get(url, param)
print(r)

https://moon.bao.ac.cn/proSearch!proSearchList.search
{
    _dc=1615632080475
    pageNo=2
    pageSize=10
    inputJsonString={"globalLang":null,"DM_DTYPE":"SCI","DM_TASKID":"CE4","DM_ZHNAME":"LCAM","DM_GRADE":"2A","DM_CBTIMEStart":"2019-01-03 00:00:00","DM_CBTIMEend":"2019-01-03 23:59:59","order_by":"desc","qsList":"","qsStart":"","qsEnd":"","bianhao":""}
    page=2
    start=10
    limit=10
}

<Response [200]>


返回两部分数据：

- `returndata`包含了本页查询的条目的所有信息，其中有保存路径，可以通过保存路径获得下载地址。
- `totalCount`本次查询命中的全部条目的数量。

In [3]:
#print(atool.pretty(r.json()))      # 打印所有数据
print('数据总条目：', r.json()['totalCount'])
for data in r.json()['returndata']:
    print(atool.pretty(data))

数据总条目： 10882
{
    dmTtype=SCI
    dmSize=1048576
    image=images/searchOrder/feiji/defaultPic.png.jpg
    paragraph=[
        {
            value=CE4
            key=探测任务
        }
        {
            value=LCAM-1
            key=有效载荷
        }
        {
            value=2A
            key=级别
        }
        {
            value=1.00(MB)
            key=大小
        }
        {
            value=A
            key=版本
        }

    ]
    dmGrade=2A
    dmQs=1
    imagePath=CE4_GRAS_LCAM-1-5350_SCI_N_20190103022913_20190103022913_0001_A.jpg
    author=GRAS
    dmTaskid=CE4
    title=CE4_GRAS_LCAM-1-5350_SCI_N_20190103022913_20190103022913_0001_A.2A
    hoverValues=[
        {
            value=CE4
            key=探测任务
        }
        {
            value=LCAM-1
            key=有效载荷
        }
        {
            value=2A
            key=级别
        }
        {
            value=1.00(MB)
            key=大小
        }
        {
            value=PDS
            key=数据格式
        }
     

由上可知，每个data数据条目中，`dmStorepath`即为数据文件真正的保存地址。

将`/PUBDATA/`替换为`http://moon.bao.ac.cn/cedownload/`即可获得真正的下载链接。

In [4]:
print('数据总条目：', r.json()['totalCount'])
for data in r.json()['returndata']:
    # 真实的下载链接
    print(data['dmStorepath'].replace('/PUBDATA/', 'http://moon.bao.ac.cn/cedownload/'))

数据总条目： 10882
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LCAM-1-5350_SCI_N_20190103022913_20190103022913_0001_A.2A
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LCAM-1-5350_SCI_N_20190103022913_20190103022913_0001_A.2AL
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LCAM-1-5351_SCI_N_20190103022913_20190103022913_0001_A.2A
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LCAM-1-5351_SCI_N_20190103022913_20190103022913_0001_A.2AL
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LCAM-1-5352_SCI_N_20190103022913_20190103022913_0001_A.2A
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LCAM-1-5352_SCI_N_20190103022913_20190103022913_0001_A.2AL
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LCAM-1-5353_SCI_N_20190103022913_20190103022913_0001_A.2A
http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/LCAM/1/2A/2019-03/CE4_GRAS_LC

## 2. 设计`get_ChangE_download_link()`函数

通过自定义的查询获得嫦娥公开科学数据的下载链接。

根据上面的内容，可以设计一个网络爬虫程序，根据给定的`页码`、`页面大小`、`任务`、`数据类型`、`载荷类型`、`数据级别`、`开始时间`、`结束时间`向服务器发起查询，并根据查询结果组装真正的下载链接。

目前支持嫦娥1～嫦娥5的数据查询。

In [1]:
import requests
import time
import atool

def get_ChangE_download_link(pageNo, pageSize, task_id='CE4', data_type='SCI', payload='PCAM', data_grade='2B', date_start='default', date_end='today'):
    # 默认情况下，根据任务设置查询的起始时间
    if date_start == 'default':
        if task_id == 'CE5':
            date_start = '2020-12-01 00:00:00'
        if task_id == 'CE4':
            date_start = '2019-01-04 00:00:00'
        if task_id == 'CE3':
            date_start = '2009-12-31 00:00:00'
        if task_id == 'CE2':
            date_start = '2010-10-01 00:00:00'
        if task_id == 'CE1':
            date_start = '2007-10-24 00:00:00'

    # 默认情况下，设置查询的结束时间为当天
    if date_end == 'today':
        date_end = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    # 组装参数
    url = 'https://' + 'moon.bao.ac.cn' + '/proSearch!proSearchList.search'
    inputJsonString = '{"globalLang":null,"DM_DTYPE":"%s","DM_TASKID":"%s","DM_ZHNAME":"%s","DM_GRADE":"%s","DM_CBTIMEStart":"%s","DM_CBTIMEend":"%s","order_by":"desc","qsList":"","qsStart":"","qsEnd":"","bianhao":""}' % (data_type, task_id, payload, data_grade, date_start, date_end)
    param = {
        '_dc': '1615623500055', 
        'pageNo': str(pageNo), 
        'pageSize': str(pageSize), 
        'inputJsonString': inputJsonString, 
        'page': str(pageNo), 
        'start': str((pageNo-1)*pageSize), 
        'limit': str(pageSize)}
        
    # 访问网站，查询
    r = requests.get(url, param)
    if r.ok:
        link = []
        link.append(r.json()['totalCount'])
        for data in r.json()['returndata']:
            #print(atool.pretty(data))
            # 真实的下载链接
            link.append(data['dmStorepath'].replace('/PUBDATA/', 'http://moon.bao.ac.cn/cedownload/'))

        return link
    else:
        return None

下面是几个使用的例子。

In [11]:
# 默认查询嫦娥4的数据，默认载荷是PCAM，默认数据级别2B
links = get_ChangE_download_link(1, 10, date_start='2019-12-04 00:00:00', date_end='today')
if links != None:
    print(atool.pretty(links))

[
    1684
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2020-04/CE4_GRAS_PCAML-C-011_SCI_N_20200329101518_20200329101518_0124_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2020-04/CE4_GRAS_PCAML-C-011_SCI_N_20200329101518_20200329101518_0124_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/C/2B/2020-04/CE4_GRAS_PCAMR-C-011_SCI_N_20200329101502_20200329101502_0124_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/C/2B/2020-04/CE4_GRAS_PCAMR-C-011_SCI_N_20200329101502_20200329101502_0124_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2020-04/CE4_GRAS_PCAML-C-010_SCI_N_20200329100955_20200329100955_0124_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2020-04/CE4_GRAS_PCAML-C-010_SCI_N_20200329100955_20200329100955_0124_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/C/2B/2020-04/CE4_GRAS_PCAMR-C-010_SCI_N_20200329100940_20200329100940_0124_B.2B
    http://moon.bao.ac.cn/cedownload/CE4R

In [10]:
# 指定任务为嫦娥2，载荷留空查所有，数据级别留空查所有
links = get_ChangE_download_link(1, 10, task_id='CE2', payload='', data_grade='')
if links != None:
    print(atool.pretty(links))

[
    38843
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522233204_20110523012536_2771_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522213831_20110522233203_2770_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522194457_20110522213830_2769_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522175124_20110522194456_2768_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522155751_20110522175123_2767_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522140418_20110522155750_2766_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522121045_20110522140417_2765_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522101712_20110522121044_2764_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_P_20110522082339_20110522101711_2763_A.2C
    http://moon.bao.ac.cn/cedownload/HPD/2C/CE2_BMYK_HPD_SCI_

In [12]:
# 指定任务为嫦娥5，载荷为LMS，数据级别留空查所有
links = get_ChangE_download_link(1, 20, task_id='CE5', payload='LMS', data_grade='')
if links != None:
    print(atool.pretty(links))

[
    2543
    http://moon.bao.ac.cn/cedownload/PUBDATA/CE5ROLL/LMS/LMS-C-C000/2B/202104/CE5-L_GRAS_LMS-C-C000_AUX_N_20201202165629_20201202170410_0019_A.2B
    http://moon.bao.ac.cn/cedownload/CE5ROLL/LMS/LMS-C-C000/2B/202104/CE5-L_GRAS_LMS-C-C000_SCI_N_20201202165629_20201202170410_0019_A.2B
    http://moon.bao.ac.cn/cedownload/PUBDATA/CE5ROLL/LMS/LMS-C-C000/2B/202104/CE5-L_GRAS_LMS-C-C000_SCI_N_20201202165629_20201202170410_0019_A.2BL
    http://moon.bao.ac.cn/cedownload/CE5ROLL/LMS/LMS-C-C000/2B/202104/CE5-L_GRAS_LMS-C-C000_SCI_N_20201202165629_20201202170410_0019_A.2BL
    http://moon.bao.ac.cn/cedownload/CE5ROLL/LMS/LMS-M-C000/2B/202104/CE5-L_GRAS_LMS-M-C000_SCI_N_20201202165629_20201202170410_0019_A.2B
    http://moon.bao.ac.cn/cedownload/PUBDATA/CE5ROLL/LMS/LMS-M-C000/2B/202104/CE5-L_GRAS_LMS-M-C000_SCI_N_20201202165629_20201202170410_0019_A.2BL
    http://moon.bao.ac.cn/cedownload/CE5ROLL/LMS/LMS-M-C000/2B/202104/CE5-L_GRAS_LMS-M-C000_SCI_N_20201202165629_20201202170410_0019_A

## 3.1 下载数据（使用wget）

- 第一步：通过`get_ChangE_download_link()`获取下载链接；
- 第二步：将链接写入文本文件；
- 第三步：用`wget`批量下载，当然也可以写一个函数，用`requests`下载。

In [15]:
# 默认查询嫦娥4的数据，默认载荷是PCAM，默认数据级别2B
links = get_ChangE_download_link(27, 100)
if links != None:
    print(atool.pretty(links))

[
    7268
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-004_SCI_N_20191103012713_20191103012713_0094_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-004_SCI_N_20191103012713_20191103012713_0094_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-004_SCI_N_20191103012707_20191103012707_0094_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-004_SCI_N_20191103012707_20191103012707_0094_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-003_SCI_N_20191103012553_20191103012553_0094_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-003_SCI_N_20191103012553_20191103012553_0094_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-003_SCI_N_20191103012547_20191103012547_0094_B.2B
    http://moon.bao.ac.cn/cedownload/CE4R

In [12]:
# 将下载链接写入文件
print('写入下载链接', end='')
file_handle=open('download.txt',mode='w')
for l in links:
    if isinstance(l, str):
        file_handle.write(l+'\n')
        print('.', end='')
file_handle.close()
print('完成')

写入下载链接....................................................................................................完成


In [13]:
# 将数据下载到data目录中
!mkdir data
!wget -P data -nv -i download.txt

mkdir: 无法创建目录 “data”: 文件已存在
2021-05-07 15:12:28 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-029_SCI_N_20191103022605_20191103022605_0094_B.2B [2032128/2032128] -> "data/CE4_GRAS_PCAML-Q-029_SCI_N_20191103022605_20191103022605_0094_B.2B.1" [1]
2021-05-07 15:12:28 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-029_SCI_N_20191103022605_20191103022605_0094_B.2BL [6779/6779] -> "data/CE4_GRAS_PCAML-Q-029_SCI_N_20191103022605_20191103022605_0094_B.2BL.1" [1]
2021-05-07 15:12:28 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-029_SCI_N_20191103022600_20191103022600_0094_B.2B [2032128/2032128] -> "data/CE4_GRAS_PCAMR-Q-029_SCI_N_20191103022600_20191103022600_0094_B.2B.1" [1]
2021-05-07 15:12:29 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-029_SCI_N_20191103022600_20191103022600_0094_B.2BL [6780/6780] -> "data/CE4_GRAS_PCAMR-Q-029_SCI_N_

2021-05-07 15:12:40 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-021_SCI_N_20191103020243_20191103020243_0094_B.2BL [6768/6768] -> "data/CE4_GRAS_PCAML-Q-021_SCI_N_20191103020243_20191103020243_0094_B.2BL.1" [1]
2021-05-07 15:12:40 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-021_SCI_N_20191103020237_20191103020237_0094_B.2B [2032128/2032128] -> "data/CE4_GRAS_PCAMR-Q-021_SCI_N_20191103020237_20191103020237_0094_B.2B.1" [1]
2021-05-07 15:12:40 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-021_SCI_N_20191103020237_20191103020237_0094_B.2BL [6769/6769] -> "data/CE4_GRAS_PCAMR-Q-021_SCI_N_20191103020237_20191103020237_0094_B.2BL.1" [1]
2021-05-07 15:12:41 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-020_SCI_N_20191103020123_20191103020123_0094_B.2B [2032128/2032128] -> "data/CE4_GRAS_PCAML-Q-020_SCI_N_20191103020123_201911030201

2021-05-07 15:12:53 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-013_SCI_N_20191103013908_20191103013908_0094_B.2B [2032128/2032128] -> "data/CE4_GRAS_PCAMR-Q-013_SCI_N_20191103013908_20191103013908_0094_B.2B" [1]
2021-05-07 15:12:53 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-013_SCI_N_20191103013908_20191103013908_0094_B.2BL [6778/6778] -> "data/CE4_GRAS_PCAMR-Q-013_SCI_N_20191103013908_20191103013908_0094_B.2BL" [1]
2021-05-07 15:12:54 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-012_SCI_N_20191103013753_20191103013753_0094_B.2B [2032128/2032128] -> "data/CE4_GRAS_PCAML-Q-012_SCI_N_20191103013753_20191103013753_0094_B.2B" [1]
2021-05-07 15:12:54 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/Q/2B/2019-11/CE4_GRAS_PCAML-Q-012_SCI_N_20191103013753_20191103013753_0094_B.2BL [6779/6779] -> "data/CE4_GRAS_PCAML-Q-012_SCI_N_20191103013753_20191103013753_0094

2021-05-07 15:13:05 URL:https://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/Q/2B/2019-11/CE4_GRAS_PCAMR-Q-005_SCI_N_20191103012827_20191103012827_0094_B.2BL [6777/6777] -> "data/CE4_GRAS_PCAMR-Q-005_SCI_N_20191103012827_20191103012827_0094_B.2BL" [1]
下载完毕 --2021-05-07 15:13:05--
总用时：38s
下载了：100 个文件，34s (2.90 MB/s) 中的 97M


## 3.2 下载数据（使用requests）

In [2]:
# 使用requests下载
import requests

import os

def is_datafile_exist(filename, filebuf=None, path='.', method='name'):
    """
    检查给定的文件是否已经存在
    目前只比对文件名
    """
    if method == 'name':
        for f in os.listdir(path):
            if f[0] == '.':
                pass
            else:
                if filename == f:
                    return True
        return False
    elif method == 'size':
        pass
    elif method == 'content':
        pass

def download(url, path):
    """
    将给定的url下载到path中
    """
    def pleasant_size(size):
        if size > 1024:
            size = size/1024
            [rs, unit] = pleasant_size(size)
            if unit == 'B':
                return [rs, 'kB']
            elif unit == 'kB':
                return [rs, 'MB']
            elif unit == 'MB':
                return [rs, 'GB']
            elif unit == 'TB':
                return [rs, 'TB']
            elif unit == 'PB':
                return [rs, 'PB']
        else:
            return [size, 'B']
        
        
    filename = url[(url.rfind('/')+1):len(url)]
    
    r = requests.get(url, stream=True)
    
    if r.ok:
        realsize = int(r.headers['Content-Length'])
        [size, unit] = pleasant_size(realsize)
        print('开始下载 %s（%.2f %s） ' % (filename, size, unit), end='')
        finished = 0
        with open(path + '/' + filename, "wb") as data:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    data.write(chunk)
                    finished = finished + len(chunk)
                if (finished>realsize/10):
                    print('.', end='')
                    finished=0
        data.close()
        print('完成')
    else:
        print(r, '连接失败')

首先获取下载链接。

In [4]:
# 默认查询嫦娥4的数据，默认载荷是PCAM，默认数据级别2B
links = get_ChangE_download_link(73, 100)
print(len(links))
if links != None:
    print(atool.pretty(links))

69
[
    7268
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2019-07/CE4_GRAS_PCAML-C-009_SCI_N_20190111034644_20190111034644_0002_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2019-07/CE4_GRAS_PCAML-C-009_SCI_N_20190111034644_20190111034644_0002_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/C/2B/2019-07/CE4_GRAS_PCAMR-C-009_SCI_N_20190111034628_20190111034628_0002_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/C/2B/2019-07/CE4_GRAS_PCAMR-C-009_SCI_N_20190111034628_20190111034628_0002_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2019-07/CE4_GRAS_PCAML-C-008_SCI_N_20190111034331_20190111034331_0002_B.2B
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAML/C/2B/2019-07/CE4_GRAS_PCAML-C-008_SCI_N_20190111034331_20190111034331_0002_B.2BL
    http://moon.bao.ac.cn/cedownload/CE4ROLL/CE4/PCAMR/C/2B/2019-07/CE4_GRAS_PCAMR-C-008_SCI_N_20190111034316_20190111034316_0002_B.2B
    http://moon.bao.ac.cn/cedownload/C

如果文件不存在（目前只判断文件名）则进行下载

In [6]:
%%time
for l in links[1:len(links)]:
    filename = l[(l.rfind('/')+1):len(l)]
    if is_datafile_exist(filename, path='data'):
        print('文件 %s 已存在' % (filename))
    else:
        download(l, 'data')
print('下载完成')

文件 CE4_GRAS_PCAML-C-009_SCI_N_20190111034644_20190111034644_0002_B.2B 已存在
文件 CE4_GRAS_PCAML-C-009_SCI_N_20190111034644_20190111034644_0002_B.2BL 已存在
文件 CE4_GRAS_PCAMR-C-009_SCI_N_20190111034628_20190111034628_0002_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-009_SCI_N_20190111034628_20190111034628_0002_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-008_SCI_N_20190111034331_20190111034331_0002_B.2B 已存在
文件 CE4_GRAS_PCAML-C-008_SCI_N_20190111034331_20190111034331_0002_B.2BL 已存在
文件 CE4_GRAS_PCAMR-C-008_SCI_N_20190111034316_20190111034316_0002_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-008_SCI_N_20190111034316_20190111034316_0002_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-007_SCI_N_20190111034211_20190111034211_0002_B.2B 已存在
文件 CE4_GRAS_PCAML-C-007_SCI_N_20190111034211_20190111034211_0002_B.2BL 已存在
文件 CE4_GRAS_PCAMR-C-007_SCI_N_20190111034156_20190111034156_0002_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-007_SCI_N_20190111034156_20190111034156_0002_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-006_SCI_N_20190111034051_20190111034051_0002_B.2B 已存在
文件 CE4_GRAS_PCAML-C-006_SCI_N_20

## 4. 一次性下载全部数据文件

网页端锁死了页面尺寸，一次只能下载10个数据文件。如果查询返回的数据文件数目为几千个，一页一页的下载就不太方便。

使用requests可以将页面尺寸作为参数传递给GET，即可以根据需要调整页面尺寸，但是我也仅测试过每页100个条目，更大页面尺寸是否可行并没有测试过。

不过既然可以通过查询知道确切的数据文件的总数目，所以只需要简单的用循环自动翻页，多次“查询-下载”即可一次性下载全部的数据文件了。

In [3]:
%%time

import time

# 默认查询嫦娥4的数据，默认载荷是PCAM，默认数据级别2B
# 查第0页，不会返回具体数据，但是会返回数据文件的数目
links = get_ChangE_download_link(0, 10)
if links != None:
    totalcount = int(links[0])
    print('查询到数据文件共 %d 个' % (totalcount))
    
    print('开始下载...')
    for i in range(int(totalcount/10+1)):
        print(i+1)
        links = get_ChangE_download_link(i+1, 10)
        if links != None:
                for l in links[1:len(links)]:
                    filename = l[(l.rfind('/')+1):len(l)]
                    if is_datafile_exist(filename, path='data'):
                        print('文件 %s 已存在' % (filename))
                    else:
                        download(l, 'data')
                print('下载完成')
                time.sleep(0.5)
    

查询到数据文件共 7268 个
开始下载...
1
文件 CE4_GRAS_PCAML-C-011_SCI_N_20200428095013_20200428095013_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-011_SCI_N_20200428095013_20200428095013_0128_B.2BL 已存在
文件 CE4_GRAS_PCAMR-C-011_SCI_N_20200428094957_20200428094957_0128_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-011_SCI_N_20200428094957_20200428094957_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-010_SCI_N_20200428094844_20200428094844_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-010_SCI_N_20200428094844_20200428094844_0128_B.2BL 已存在
文件 CE4_GRAS_PCAMR-C-010_SCI_N_20200428094828_20200428094828_0128_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-010_SCI_N_20200428094828_20200428094828_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-009_SCI_N_20200428094715_20200428094715_0128_B.2B 已存在
文件 CE4_GRAS_PCAML-C-009_SCI_N_20200428094715_20200428094715_0128_B.2BL 已存在
下载完成
2
文件 CE4_GRAS_PCAMR-C-009_SCI_N_20200428094659_20200428094659_0128_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-009_SCI_N_20200428094659_20200428094659_0128_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-008_SCI_N_20200428094546_20200428094546_0128_B.2B 已存在

12
文件 CE4_GRAS_PCAMR-Q-082_SCI_N_20200419124450_20200419124450_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-082_SCI_N_20200419124450_20200419124450_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-081_SCI_N_20200419124335_20200419124335_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-081_SCI_N_20200419124335_20200419124335_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-081_SCI_N_20200419124330_20200419124330_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-081_SCI_N_20200419124330_20200419124330_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-080_SCI_N_20200419124215_20200419124215_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-080_SCI_N_20200419124215_20200419124215_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-080_SCI_N_20200419124210_20200419124210_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-080_SCI_N_20200419124210_20200419124210_0126_B.2BL 已存在
下载完成
13
文件 CE4_GRAS_PCAML-Q-079_SCI_N_20200419123935_20200419123935_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-079_SCI_N_20200419123935_20200419123935_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-079_SCI_N_20200419123930_20200419123930_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-0

23
文件 CE4_GRAS_PCAML-Q-054_SCI_N_20200419101245_20200419101245_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-054_SCI_N_20200419101245_20200419101245_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-054_SCI_N_20200419101240_20200419101240_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-054_SCI_N_20200419101240_20200419101240_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-053_SCI_N_20200419101125_20200419101125_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-053_SCI_N_20200419101125_20200419101125_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-053_SCI_N_20200419101120_20200419101120_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-053_SCI_N_20200419101120_20200419101120_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-052_SCI_N_20200419101005_20200419101005_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-052_SCI_N_20200419101005_20200419101005_0126_B.2BL 已存在
下载完成
24
文件 CE4_GRAS_PCAMR-Q-052_SCI_N_20200419101000_20200419101000_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-052_SCI_N_20200419101000_20200419101000_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-051_SCI_N_20200419100845_20200419100845_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-0

34
文件 CE4_GRAS_PCAMR-Q-027_SCI_N_20200419090705_20200419090705_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-027_SCI_N_20200419090705_20200419090705_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-026_SCI_N_20200419090550_20200419090550_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-026_SCI_N_20200419090550_20200419090550_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-026_SCI_N_20200419090545_20200419090545_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-026_SCI_N_20200419090545_20200419090545_0126_B.2BL 已存在
文件 CE4_GRAS_PCAML-Q-025_SCI_N_20200419090430_20200419090430_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-025_SCI_N_20200419090430_20200419090430_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-025_SCI_N_20200419090425_20200419090425_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-025_SCI_N_20200419090425_20200419090425_0126_B.2BL 已存在
下载完成
35
文件 CE4_GRAS_PCAML-Q-024_SCI_N_20200419090310_20200419090310_0126_B.2B 已存在
文件 CE4_GRAS_PCAML-Q-024_SCI_N_20200419090310_20200419090310_0126_B.2BL 已存在
文件 CE4_GRAS_PCAMR-Q-024_SCI_N_20200419090305_20200419090305_0126_B.2B 已存在
文件 CE4_GRAS_PCAMR-Q-0

45
文件 CE4_GRAS_PCAML-C-011_SCI_N_20200329101518_20200329101518_0124_B.2B 已存在
文件 CE4_GRAS_PCAML-C-011_SCI_N_20200329101518_20200329101518_0124_B.2BL 已存在
文件 CE4_GRAS_PCAMR-C-011_SCI_N_20200329101502_20200329101502_0124_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-011_SCI_N_20200329101502_20200329101502_0124_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-010_SCI_N_20200329100955_20200329100955_0124_B.2B 已存在
文件 CE4_GRAS_PCAML-C-010_SCI_N_20200329100955_20200329100955_0124_B.2BL 已存在
文件 CE4_GRAS_PCAMR-C-010_SCI_N_20200329100940_20200329100940_0124_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-010_SCI_N_20200329100940_20200329100940_0124_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-009_SCI_N_20200329052104_20200329052104_0124_B.2B 已存在
文件 CE4_GRAS_PCAML-C-009_SCI_N_20200329052104_20200329052104_0124_B.2BL 已存在
下载完成
46
文件 CE4_GRAS_PCAMR-C-009_SCI_N_20200329052049_20200329052049_0124_B.2B 已存在
文件 CE4_GRAS_PCAMR-C-009_SCI_N_20200329052049_20200329052049_0124_B.2BL 已存在
文件 CE4_GRAS_PCAML-C-008_SCI_N_20200329051935_20200329051935_0124_B.2B 已存在
文件 CE4_GRAS_PCAML-C-0

...

开始下载 CE4_GRAS_PCAML-Q-015_SCI_N_20190131175211_20190131175211_0005_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-015_SCI_N_20190131175211_20190131175211_0005_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-015_SCI_N_20190131175206_20190131175206_0005_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-015_SCI_N_20190131175206_20190131175206_0005_B.2BL（6.62 kB） ......完成
下载完成
687
开始下载 CE4_GRAS_PCAML-Q-014_SCI_N_20190131175052_20190131175052_0005_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-014_SCI_N_20190131175052_20190131175052_0005_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-014_SCI_N_20190131175047_20190131175047_0005_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-014_SCI_N_20190131175047_20190131175047_0005_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAML-Q-013_SCI_N_20190131174229_20190131174229_0005_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-013_SCI_N_20190131174229_20190131174229_0005_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-013_SCI_N_20190131174223_20190131174223_0005_B.2B

开始下载 CE4_GRAS_PCAML-Q-061_SCI_N_20190112093204_20190112093204_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-061_SCI_N_20190112093204_20190112093204_0003_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-061_SCI_N_20190112093158_20190112093158_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-061_SCI_N_20190112093158_20190112093158_0003_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAML-Q-060_SCI_N_20190112093044_20190112093044_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-060_SCI_N_20190112093044_20190112093044_0003_B.2BL（6.61 kB） ......完成
下载完成
696
开始下载 CE4_GRAS_PCAMR-Q-060_SCI_N_20190112093038_20190112093038_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-060_SCI_N_20190112093038_20190112093038_0003_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAML-Q-059_SCI_N_20190112092924_20190112092924_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-059_SCI_N_20190112092924_20190112092924_0003_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-059_SCI_N_20190112092918_20190112092918_0003_B.2B

开始下载 CE4_GRAS_PCAML-Q-039_SCI_N_20190112085429_20190112085429_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-039_SCI_N_20190112085429_20190112085429_0003_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-039_SCI_N_20190112085424_20190112085424_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-039_SCI_N_20190112085424_20190112085424_0003_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAML-Q-038_SCI_N_20190112085309_20190112085309_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-038_SCI_N_20190112085309_20190112085309_0003_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-038_SCI_N_20190112085304_20190112085304_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-038_SCI_N_20190112085304_20190112085304_0003_B.2BL（6.62 kB） ......完成
下载完成
705
开始下载 CE4_GRAS_PCAML-Q-037_SCI_N_20190112085149_20190112085149_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-037_SCI_N_20190112085149_20190112085149_0003_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-037_SCI_N_20190112085144_20190112085144_0003_B.2B

开始下载 CE4_GRAS_PCAML-Q-017_SCI_N_20190112075710_20190112075710_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-017_SCI_N_20190112075710_20190112075710_0003_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-017_SCI_N_20190112075704_20190112075704_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-017_SCI_N_20190112075704_20190112075704_0003_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAML-Q-016_SCI_N_20190112075550_20190112075550_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-016_SCI_N_20190112075550_20190112075550_0003_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAMR-Q-016_SCI_N_20190112075544_20190112075544_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAMR-Q-016_SCI_N_20190112075544_20190112075544_0003_B.2BL（6.62 kB） ......完成
开始下载 CE4_GRAS_PCAML-Q-015_SCI_N_20190112075430_20190112075430_0003_B.2B（1.94 MB） .........完成
开始下载 CE4_GRAS_PCAML-Q-015_SCI_N_20190112075430_20190112075430_0003_B.2BL（6.62 kB） ......完成
下载完成
714
开始下载 CE4_GRAS_PCAMR-Q-015_SCI_N_20190112075424_20190112075424_0003_B.2B

开始下载 CE4_GRAS_PCAML-C-007_SCI_N_20190111034211_20190111034211_0002_B.2B（7.75 MB） .........完成
开始下载 CE4_GRAS_PCAML-C-007_SCI_N_20190111034211_20190111034211_0002_B.2BL（6.61 kB） ......完成
下载完成
722
开始下载 CE4_GRAS_PCAMR-C-007_SCI_N_20190111034156_20190111034156_0002_B.2B（7.75 MB） .........完成
开始下载 CE4_GRAS_PCAMR-C-007_SCI_N_20190111034156_20190111034156_0002_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAML-C-006_SCI_N_20190111034051_20190111034051_0002_B.2B（7.75 MB） .........完成
开始下载 CE4_GRAS_PCAML-C-006_SCI_N_20190111034051_20190111034051_0002_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-C-006_SCI_N_20190111034036_20190111034036_0002_B.2B（7.75 MB） .........完成
开始下载 CE4_GRAS_PCAMR-C-006_SCI_N_20190111034036_20190111034036_0002_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAML-C-005_SCI_N_20190110212053_20190110212053_0002_B.2B（7.75 MB） .........完成
开始下载 CE4_GRAS_PCAML-C-005_SCI_N_20190110212053_20190110212053_0002_B.2BL（6.61 kB） ......完成
开始下载 CE4_GRAS_PCAMR-C-005_SCI_N_20190110212037_20190110212037_0002_B.2B